# YOLOv8n Object Detection Keras Model - Quantization for IMX500

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/imx500_notebooks/keras/keras_yolov8n_for_imx500.ipynb)

## Overview

In this tutorial, we will illustrate a basic and quick process of preparing a pre-trained model for deployment using MCT. Specifically, we will demonstrate how to download a pre-trained YOLOv8n model from the MCT Models Library, compress it, and make it deployment-ready using MCT's post-training quantization techniques. Additionally, we will demonstrate an optional optimization step using MCT's Gradient-Based Post Training Quantization.

We will use an existing pre-trained YOLOv8n model based on [Ultralytics](https://github.com/ultralytics/ultralytics). The model was slightly adjusted with integrated NMS layer. We will quantize the model using MCT post training quantization and evaluate the performance of the floating point model and the quantized model on COCO dataset.


## Summary

In this tutorial we will cover:

1. Post-Training Quantization using MCT of Keras object detection model.
2. An optional optimization step of Gradient-Based Post Training Quantization. 
3. Data preparation - loading and preprocessing validation and representative datasets from COCO.
4. Accuracy evaluation of the floating-point and the quantized models.

## Setup
### Install the relevant packages

In [ ]:
TF_VER = '2.14.0'

!pip install -q tensorflow=={TF_VER}
!pip install -q pycocotools
!pip install 'huggingface-hub<=0.21.4'

Install MCT (if it’s not already installed). Additionally, in order to use all the necessary utility functions for this tutorial, we also copy [MCT tutorials folder](https://github.com/sony/model_optimization/tree/main/tutorials) and add it to the system path.

In [ ]:
import sys
import os
import importlib

if not importlib.util.find_spec('model_compression_toolkit'):
    !pip install model_compression_toolkit
!git clone https://github.com/sony/model_optimization.git temp_mct && mv temp_mct/tutorials . && \rm -rf temp_mct
sys.path.insert(0,"tutorials")

### Download COCO evaluation set

In [ ]:
if not os.path.isdir('coco'):
    !wget -nc http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    !unzip -q -o annotations_trainval2017.zip -d ./coco
    !echo Done loading annotations
    !wget -nc http://images.cocodataset.org/zips/val2017.zip
    !unzip -q -o val2017.zip -d ./coco
    !echo Done loading val2017 images

## Model Quantization

### Download a Pre-Trained Model 

We begin by loading a pre-trained [YOLOv8n](https://huggingface.co/SSI-DNN/test_keras_yolov8n_640x640) model. This implementation is based on [Ultralytics](https://github.com/ultralytics/ultralytics) and includes a slightly modified version of yolov8 detection-head (mainly the box decoding part) that was adapted for model quantization. For further insights into the model's implementation details, please refer to [MCT Models Library - yolov8](https://github.com/sony/model_optimization/tree/main/tutorials/mct_model_garden/models_keras/yolov8).   

In [ ]:
from huggingface_hub import from_pretrained_keras

model = from_pretrained_keras('SSI-DNN/keras_yolov8n_640x640_pp')

### Post training quantization using Model Compression Toolkit 

Now, we're all set to use MCT's post-training quantization. To begin, we'll define a representative dataset and proceed with the model quantization. Please note that, for demonstration purposes, we'll use the evaluation dataset as our representative dataset. We'll calibrate the model using 100 representative images, divided into 20 iterations of 'batch_size' images each. 

Additionally, to further compress the model's memory footprint, we will employ the mixed-precision quantization technique. This method allows each layer to be quantized with different precision options: 2, 4, and 8 bits, aligning with the imx500 target platform capabilities.  

In [ ]:
import model_compression_toolkit as mct
from tutorials.mct_model_garden.evaluation_metrics.coco_evaluation import CocoDataset, DataLoader
from tutorials.mct_model_garden.models_keras.yolov8.yolov8_preprocess import yolov8_preprocess
from typing import Iterator, Tuple, List

REPRESENTATIVE_DATASET_FOLDER = './coco/val2017/'
REPRESENTATIVE_DATASET_ANNOTATION_FILE = './coco/annotations/instances_val2017.json'
BATCH_SIZE = 5
n_iters = 20

# Load representative dataset
representative_dataset = CocoDataset(dataset_folder=REPRESENTATIVE_DATASET_FOLDER,
                                     annotation_file=REPRESENTATIVE_DATASET_ANNOTATION_FILE,
                                     preprocess=yolov8_preprocess)

rep_data_loader = DataLoader(representative_dataset, BATCH_SIZE, shuffle=True)

# Define representative dataset generator
def get_representative_dataset(n_iter: int, dataset_loader: DataLoader):
    """
    This function creates a representative dataset generator. The generator yields numpy
        arrays of batches of shape: [Batch, H, W ,C].
    Args:
        n_iter: number of iterations for MCT to calibrate on
        dataset_loader: an iterable DataLoader 
    Returns:
        A representative dataset generator
    """
    def representative_dataset():
        ds_iter = iter(dataset_loader)
        for _ in range(n_iter):
            yield [next(ds_iter)[0]]

    return representative_dataset

# Get representative dataset generator
representative_dataset_gen = get_representative_dataset(n_iters, rep_data_loader)

# Set IMX500-v1 TPC
tpc = mct.get_target_platform_capabilities("tensorflow", 'imx500', target_platform_version='v1')

# Specify the necessary configuration for mixed precision quantization. 
mp_config = mct.core.MixedPrecisionQuantizationConfig(num_of_images=5, use_hessian_based_scores=False)

# Specify the core configuration. Enable "shift negative" correction for better non-linear activation quantization.
config = mct.core.CoreConfig(mixed_precision_config=mp_config,
                             quantization_config=mct.core.QuantizationConfig(shift_negative_activation_correction=True))

# Define target Resource Utilization for mixed precision weights quantization (76% of 'standard' 8bits quantization)
resource_utilization_data = mct.core.keras_resource_utilization_data(model,
                                   representative_dataset_gen,
                                   config,
                                   target_platform_capabilities=tpc)
resource_utilization = mct.core.ResourceUtilization(resource_utilization_data.weights_memory * 0.76)

# Perform post training quantization
quant_model, _ = mct.ptq.keras_post_training_quantization(model,
                                                          representative_dataset_gen,
                                                          target_resource_utilization=resource_utilization,
                                                          core_config=config,
                                                          target_platform_capabilities=tpc)
print('Quantized model is ready')

### Model Export

Now, we can export the quantized model, ready for deployment, into a `.keras` format file. Please ensure that the `save_model_path` has been set correctly. 

In [ ]:
mct.exporter.keras_export_model(model=quant_model, save_model_path='./qmodel.keras')


### Gradient-Based Post Training Quantization using Model Compression Toolkit
Here we demonstrate how to further optimize the quantized model performance using gradient-based PTQ technique.
**Please note that this section is computationally heavy, and it's recommended to run it on a GPU. For fast deployment, you may choose to skip this step.** 

We will start by loading the COCO training set, and re-define the representative dataset accordingly. 

In [ ]:
if not os.path.isdir('coco/train2017'):
    !wget -nc http://images.cocodataset.org/zips/train2017.zip
    !unzip -q -o train2017.zip -d ./coco
    !echo Done loading train2017 images

REPRESENTATIVE_DATASET_FOLDER = './coco/train2017/'
REPRESENTATIVE_DATASET_ANNOTATION_FILE = './coco/annotations/instances_train2017.json'
BATCH_SIZE = 5
n_iters = 20

# Load representative dataset
representative_dataset = CocoDataset(dataset_folder=REPRESENTATIVE_DATASET_FOLDER,
                                     annotation_file=REPRESENTATIVE_DATASET_ANNOTATION_FILE,
                                     preprocess=yolov8_preprocess)

rep_data_loader = DataLoader(representative_dataset, BATCH_SIZE, shuffle=True)

# Get representative dataset generator
representative_dataset_gen = get_representative_dataset(n_iters, rep_data_loader)

Next, we'll set up the Gradient-Based PTQ configuration and execute the necessary MCT command. Keep in mind that this step can be time-consuming, depending on your runtime.

In [ ]:
# Specify the necessary configuration for Gradient-Based PTQ.
n_gptq_epochs = 1000
gptq_config = mct.gptq.get_keras_gptq_config(n_epochs=n_gptq_epochs, use_hessian_based_weights=False)

# Perform Gradient-Based Post Training Quantization
quant_model, _ = mct.gptq.keras_gradient_post_training_quantization(
    model,
    representative_dataset_gen,
    target_resource_utilization=resource_utilization,
    gptq_config=gptq_config,
    core_config=config,
    target_platform_capabilities=tpc)

print('Quantized model is ready')

## Evaluation on COCO dataset

### Floating point model evaluation
Next, we evaluate the floating point model by using `cocoeval` library alongside additional dataset utilities. We can verify the mAP accuracy aligns with that of the original model. 
Note that we set the "batch_size" to 5 and the preprocessing according to [Ultralytics](https://github.com/ultralytics/ultralytics).
Please ensure that the dataset path has been set correctly before running this code cell.

In [ ]:
from tutorials.mct_model_garden.evaluation_metrics.coco_evaluation import coco_evaluate

EVAL_DATASET_FOLDER = './coco/val2017'
EVAL_DATASET_ANNOTATION_FILE = './coco/annotations/instances_val2017.json'
INPUT_RESOLUTION = 640

# Define resizing information to map between the model's output and the original image dimensions
output_resize = {'shape': (INPUT_RESOLUTION, INPUT_RESOLUTION), 'aspect_ratio_preservation': True}

# Evaluate the model on coco
eval_results = coco_evaluate(model=model,
                             dataset_folder=EVAL_DATASET_FOLDER,
                             annotation_file=EVAL_DATASET_ANNOTATION_FILE,
                             preprocess=yolov8_preprocess,
                             output_resize=output_resize,
                             batch_size=BATCH_SIZE)

# Print float model mAP results
print("Float model mAP: {:.4f}".format(eval_results[0]))

### Quantized model evaluation
Lastly, we can evaluate the performance of the quantized model. There is a slight decrease in performance that can be further mitigated by either expanding the representative dataset or employing MCT's advanced quantization methods, such as GPTQ (Gradient-Based/Enhanced Post Training Quantization).

In [ ]:
# Evaluate the model on coco
eval_results = coco_evaluate(model=quant_model,
                             dataset_folder=EVAL_DATASET_FOLDER,
                             annotation_file=EVAL_DATASET_ANNOTATION_FILE,
                             preprocess=yolov8_preprocess,
                             output_resize=output_resize,
                             batch_size=BATCH_SIZE)

# Print quantized model mAP results
print("Quantized model mAP: {:.4f}".format(eval_results[0]))

\
Copyright 2024 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.